# 기상청
- 동네예보 데이터 가지고 오기
- https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15057682
____________________

**요청 메시지 명세**
- base_date(발표일자)
- Base_time(발표시각) : 0200, 0500, 0800, 1100, 1400, 1700, 2000, 2300 (1일 8회)
    - API 제공 시간(~이후) : 02:05, 05:05, 08:05, 11:05, 14:05, 17:05, 20:05, 23:05
- nx(예보지점 X좌표), ny(예보지점 Y좌표)
____________________

**응답 메시지 명세**
- category
    - SKY: 하늘상태 - 맑음: 1 / 구름많음: 3 / 흐림: 4
        - 3시간 단위로 현재 날씨상태를 응답
        - ex) 06:00, 09:00, 12:00 .... 00:00
    - PTY: 강수형태 - 없음: 0 / 비: 1 / 비/눈: 2 / 눈: 3 / 소나기: 4 / 빗방울: 5
        - 3시간 단위로 현재 강수상태를 응답
        - ex) 06:00, 09:00, 12:00 .... 00:00
    - T3H: 3시간 온도
        - 3시간 단위로 현재 온도를 응답
        - ex) 06:00, 09:00, 12:00 .... 00:00
    - TMX(낮 최고기온), TMN(아침 최저기온)
        - TMX는 하루에 15:00 한번만 응답
        - TMN는 하루에 06:00 한번만 응답
- fcstValue(예보 값) - category의 값
- fcstDate(예보일자), fcstTime(예보시각)
____________________

**호출함수**
1. findLocation - (시, 구, 동)을 파라미터로 입력받아 <font color=red>x,y값(위치)</font>을 출력한다.
2. getTempCurrent - x,y값을 파라미터로 입력받아서 현재시간 <font color=red>온도</font>를 출력한다. (3시간 단위/03:01 경우 06:00의 온도 출력)
3. getWeatherCurrent - x,y값을 파라미터로 입력받아서 현재시간 <font color=red>날씨상태</font>를 출력한다. (3시간 단위/03:01 경우 06:00의 날씨상태 출력)
4. getPrecipitationCurrent - x,y값을 파라미터로 입력받아서 현재시간 <font color=red>강수상태</font>를 출력한다. (3시간 단위/03:01 경우 06:00의 강수상태 출력)
5. getTempMaxMinCurrent - x,y값을 파라미터로 입력받아서 <font color=red>당일의 아침 최저(06:00 기준), 오후 최고(15:00 기준)의 온도</font>를 출력한다.
____________________

**출력방법**
- 1안: 매일 아침 08:05 시간에 한번 메세지를 출력 -> 음성 변환(TTS)
    - 함수 5번만 출력한다.
    - 메시지 내용: "오늘 아침 최저기온은 15.0도 이고, 낮 최고기온은 26.0도 입니다."
- 2-1안: 현재 온도와 현재 날씨를 알고 싶을 때 출력 -> 음성 변환(TTS)
    - 함수 2번만 출력한다.
    - "메세지 내용: 현재 온도는 26 도 입니다.
- 2-2안: 현재 온도와 현재 날씨를 알고 싶을 때 출력 -> 음성 변환(TTS)
    - 함수 2번과 3번을 동시에 출력한다.
    - "메세지 내용: 현재 온도는 26 도 입니다. 현재 날씨는 맑음(or구름많음or흐림) 입니다.
- 2-3안: 현재 온도와 현재 날씨를 알고 싶을 때 출력 -> 음성 변환(TTS)
    - 함수 2번과 3번과 4번을 동시에 출력한다.
    - "메세지 내용: 현재 온도는 26 도 입니다. 현재 날씨는 맑음(or구름많음or흐림) 입니다. 현재 (비 또는 눈이 안/or/소나기가/or/빗방울이) 오고 있습니다.   
____________________

**주의사항**
- 3번 함수와 4번함수를 동시에 출력할 경우 
____________________

#### **주소를 입력하여 x축과 y축 불러오는 함수 만들기**

In [227]:
import pandas as pd

def findLocation(si,goo,dong):
    # 파일 불러오기
    df = pd.read_excel('기상청18_동네예보 조회서비스_오픈API활용가이드_격자_위경도(20210401).xlsx', sheet_name = 1)
    # df.info() # df = (columns: 17, rows: 3776)

    df1 = df.filter(items=['1단계', '2단계', '3단계', '격자 X', '격자 Y']) # (시, 구, 동, x, y) columns 필터링
    df2 = df1[df1['1단계'].isin([si]) & df1['2단계'].isin([goo]) & df1['3단계'].isin([dong])]
    nx = df2.iloc[0, 3]
    ny = df2.iloc[0, 4]
    print(f"x값: {nx}, y값: {ny}") 

#### **주소를 입력하여 x축과 y축 불러오기**

In [228]:
si = input('거주하시는 "시"를 입력해주세요: ')
goo =  input('거주하시는 "구"를 입력해주세요: ')
dong = input('거주하시는 "동"를 입력해주세요: ')
findLocation(si,goo,dong)

거주하시는 "시"를 입력해주세요:  서울특별시
거주하시는 "구"를 입력해주세요:  도봉구
거주하시는 "동"를 입력해주세요:  창제1동


x값: 61, y값: 128


#### **현재 온도(T3H) 함수 만들기**

In [229]:
import requests
import json
import datetime

vilage_weather_url = "http://apis.data.go.kr/1360000/VilageFcstInfoService/getVilageFcst?"
service_key = "nEOjm%2Bh6MajezPL97GQY7jDUzDS5L2qvhY4G%2BKfjjxqYULz7folcVjjKi1Q%2BgkD8P94Wowq7l24kNayBKGmsIA%3D%3D"

def getTempCurrent(nx, ny):
    today = datetime.datetime.today()       # 오늘의 날씨를 알려주는 함수
    current_date = today.strftime("%Y%m%d") # ex) 현재 날짜가 21년 5월 13일인 경우, '20210513' 출력
    current_time = today.strftime("%H%M")   # ex) 현재 시간이 14:15 인 경우, '1415' 출력
    tomorrow_date = today + datetime.timedelta(days=1)
    tomorrow_date = tomorrow_date.strftime("%Y%m%d") # 내일 날짜
    base_time = '0800'                    # 당일 08:00 발표시작으로 고정
    
    payload = "serviceKey=" + service_key + "&" +\
            "numOfRows=100" + "&" +\
            "pageNo=1" + "&" +\
            "dataType=json" + "&" +\
            "base_date=" + current_date + "&" +\
            "base_time=" + base_time + "&" +\
            "nx=" + nx + "&" +\
            "ny=" + ny
    # print(vilage_weather_url+payload)  # API url 확인

    # 값 요청
    res = requests.get(vilage_weather_url + payload)
    # res.status_code  # 200 정상 호출
    # res.json()
    items = res.json().get('response').get('body').get('items')

    for item in items['item']:
        if item['category'] == 'T3H':
            if item['fcstDate'] == current_date and '0000' < current_time <= '0300':
                if item['fcstTime'] == '0300':
                    T3H = item['fcstValue']    
            elif item['fcstDate'] == current_date and '0300' < current_time <= '0600':
                if item['fcstTime'] == '0600':
                    T3H = item['fcstValue']
            elif item['fcstDate'] == current_date and '0600' < current_time <= '0900':
                if item['fcstTime'] == '0900':
                    T3H = item['fcstValue']
            elif item['fcstDate'] == current_date and '0900' < current_time <= '1200':
                if item['fcstTime'] == '1200':
                    T3H = item['fcstValue']
            elif item['fcstDate'] == current_date and '1200' < current_time <= '1500':
                if item['fcstTime'] == '1500':
                    T3H = item['fcstValue']
            elif item['fcstDate'] == current_date and '1500' < current_time <= '1800':
                if item['fcstTime'] == '1800':
                    T3H = item['fcstValue']
            elif item['fcstDate'] == current_date and '1800' < current_time <= '2100':
                if item['fcstTime'] == '2100':
                    T3H = item['fcstValue']
            elif item['fcstDate'] == tomorrow_date and '2100' < current_time <= '0000':
                if item['fcstTime'] == '0000':
                    T3H = item['fcstValue']
    print(f"현재 온도는 {T3H} 도 입니다.")

In [230]:
nx = '60'
ny = '128'
getTempCurrent(nx, ny)

현재 온도는 24 도 입니다.


In [216]:
payload = "serviceKey=" + service_key + "&" +\
        "numOfRows=100" + "&" +\
        "pageNo=1" + "&" +\
        "dataType=json" + "&" +\
        "base_date=" + current_date + "&" +\
        "base_time=" + '0800' + "&" +\
        "nx=" + nx + "&" +\
        "ny=" + ny

print(vilage_weather_url+payload)

http://apis.data.go.kr/1360000/VilageFcstInfoService/getVilageFcst?serviceKey=nEOjm%2Bh6MajezPL97GQY7jDUzDS5L2qvhY4G%2BKfjjxqYULz7folcVjjKi1Q%2BgkD8P94Wowq7l24kNayBKGmsIA%3D%3D&numOfRows=100&pageNo=1&dataType=json&base_date=20210513&base_time=0800&nx=60&ny=128


#### **현재 날씨상태(SKY) 함수 만들기**


In [231]:
import requests
import json
import datetime

def getWeatherCurrent(nx, ny):
    vilage_weather_url = "http://apis.data.go.kr/1360000/VilageFcstInfoService/getVilageFcst?"
    service_key = "nEOjm%2Bh6MajezPL97GQY7jDUzDS5L2qvhY4G%2BKfjjxqYULz7folcVjjKi1Q%2BgkD8P94Wowq7l24kNayBKGmsIA%3D%3D"

    today = datetime.datetime.today()       # 오늘의 날씨를 알려주는 함수
    current_date = today.strftime("%Y%m%d") # ex) 현재 날짜가 21년 5월 13일인 경우, '20210513' 출력
    current_time = today.strftime("%H%M")   # ex) 현재 시간이 14:15 인 경우, '1415' 출력
    tomorrow_date = today + datetime.timedelta(days=1)
    tomorrow_date = tomorrow_date.strftime("%Y%m%d") # 내일 날짜
    base_time = '0800'                    # 당일 08:00 발표시작으로 고정

    payload = "serviceKey=" + service_key + "&" +\
            "numOfRows=100" + "&" +\
            "pageNo=1" + "&" +\
            "dataType=json" + "&" +\
            "base_date=" + current_date + "&" +\
            "base_time=" + base_time + "&" +\
            "nx=" + nx + "&" +\
            "ny=" + ny
    # print(vilage_weather_url+payload)  # API url 확인

    # 값 요청
    res = requests.get(vilage_weather_url + payload)
    # res.status_code  # 200 정상 호출
    # res.json()
    items = res.json().get('response').get('body').get('items')

    for item in items['item']:
        if item['category'] == 'SKY':
            if item['fcstDate'] == current_date and '0000' < current_time <= '0300':
                if item['fcstTime'] == '0300':
                    SKY = item['fcstValue']    
            elif item['fcstDate'] == current_date and '0300' < current_time <= '0600':
                if item['fcstTime'] == '0600':
                    SKY = item['fcstValue']
            elif item['fcstDate'] == current_date and '0600' < current_time <= '0900':
                if item['fcstTime'] == '0900':
                    SKY = item['fcstValue']
            elif item['fcstDate'] == current_date and '0900' < current_time <= '1200':
                if item['fcstTime'] == '1200':
                    SKY = item['fcstValue']
            elif item['fcstDate'] == current_date and '1200' < current_time <= '1500':
                if item['fcstTime'] == '1500':
                    SKY = item['fcstValue']
            elif item['fcstDate'] == current_date and '1500' < current_time <= '1800':
                if item['fcstTime'] == '1800':
                    SKY = item['fcstValue']
            elif item['fcstDate'] == current_date and '1800' < current_time <= '2100':
                if item['fcstTime'] == '2100':
                    SKY = item['fcstValue']
            elif item['fcstDate'] == tomorrow_date and '2100' < current_time <= '0000':
                if item['fcstTime'] == '0000':
                    SKY = item['fcstValue']
    if SKY == '1':
        SKY_KOREAN = '맑음'
    elif SKY == '3':
        SKY_KOREAN = '구름많음'
    elif SKY == '4':
        SKY_KOREAN = '흐림'
    else:
        SKY_KOREAN = '알수없음'
    print(f"현재 날씨는 {SKY_KOREAN} 입니다.")

In [232]:
nx = '60'
ny = '128'
getWeatherCurrent(nx, ny)

현재 날씨는 맑음 입니다.


#### **현재 강수상태(PTY) 만들기**

In [233]:
import requests
import json
import datetime

def getPrecipitationCurrent(nx, ny): # precipitation: 강수상태 (비, 눈, 이슬 등)
    vilage_weather_url = "http://apis.data.go.kr/1360000/VilageFcstInfoService/getVilageFcst?"
    service_key = "nEOjm%2Bh6MajezPL97GQY7jDUzDS5L2qvhY4G%2BKfjjxqYULz7folcVjjKi1Q%2BgkD8P94Wowq7l24kNayBKGmsIA%3D%3D"

    today = datetime.datetime.today()       # 오늘의 날씨를 알려주는 함수
    current_date = today.strftime("%Y%m%d") # ex) 현재 날짜가 21년 5월 13일인 경우, '20210513' 출력
    current_time = today.strftime("%H%M")   # ex) 현재 시간이 14:15 인 경우, '1415' 출력
    tomorrow_date = today + datetime.timedelta(days=1)
    tomorrow_date = tomorrow_date.strftime("%Y%m%d") # 내일 날짜
    base_time = '0800'                    # 당일 08:00 발표시작으로 고정

    payload = "serviceKey=" + service_key + "&" +\
            "numOfRows=100" + "&" +\
            "pageNo=1" + "&" +\
            "dataType=json" + "&" +\
            "base_date=" + current_date + "&" +\
            "base_time=" + base_time + "&" +\
            "nx=" + nx + "&" +\
            "ny=" + ny
    # print(vilage_weather_url+payload)  # API url 확인

    # 값 요청
    res = requests.get(vilage_weather_url + payload)
    # res.status_code  # 200 정상 호출
    # res.json()
    items = res.json().get('response').get('body').get('items')

    for item in items['item']:
        if item['category'] == 'PTY':
            if item['fcstDate'] == current_date and '0000' < current_time <= '0300':
                if item['fcstTime'] == '0300':
                    PTY = item['fcstValue']    
            elif item['fcstDate'] == current_date and '0300' < current_time <= '0600':
                if item['fcstTime'] == '0600':
                    PTY = item['fcstValue']
            elif item['fcstDate'] == current_date and '0600' < current_time <= '0900':
                if item['fcstTime'] == '0900':
                    PTY = item['fcstValue']
            elif item['fcstDate'] == current_date and '0900' < current_time <= '1200':
                if item['fcstTime'] == '1200':
                    PTY = item['fcstValue']
            elif item['fcstDate'] == current_date and '1200' < current_time <= '1500':
                if item['fcstTime'] == '1500':
                    PTY = item['fcstValue']
            elif item['fcstDate'] == current_date and '1500' < current_time <= '1800':
                if item['fcstTime'] == '1800':
                    PTY = item['fcstValue']
            elif item['fcstDate'] == current_date and '1800' < current_time <= '2100':
                if item['fcstTime'] == '2100':
                    PTY = item['fcstValue']
            elif item['fcstDate'] == tomorrow_date and '2100' < current_time <= '0000':
                if item['fcstTime'] == '0000':
                    PTY = item['fcstValue']
    if PTY == '1':
        PTY_KOREAN = '비가'
    elif PTY == '2':
        PTY_KOREAN = '비 또는 눈이'
    elif PTY == '3':
        PTY_KOREAN = '눈이'
    elif PTY == '4':
        PTY_KOREAN = '소나기가'
    else:
        PTY_KOREAN = '비 또는 눈이 안'
    print(f"현재 {PTY_KOREAN} 오고 있습니다.")

In [234]:
nx = '60'
ny = '128'
getPrecipitationCurrent(nx, ny)

현재 비 또는 눈이 안 오고 있습니다.


#### **TMX(낮 최고기온), TMN(아침 최저기온) 만들기**
- TMN: 매일 fcstDate 06:00 한번
- TMX: 매일 fcstDate 15:00 한번

In [235]:
import requests
import json
import datetime

def getTempMaxMinCurrent(nx, ny):
    vilage_weather_url = "http://apis.data.go.kr/1360000/VilageFcstInfoService/getVilageFcst?"
    service_key = "nEOjm%2Bh6MajezPL97GQY7jDUzDS5L2qvhY4G%2BKfjjxqYULz7folcVjjKi1Q%2BgkD8P94Wowq7l24kNayBKGmsIA%3D%3D"

    today = datetime.datetime.today()       # 오늘의 날씨를 알려주는 함수
    current_date = today.strftime("%Y%m%d") # ex) 현재 날짜가 21년 5월 13일인 경우, '20210513' 출력
    current_time = today.strftime("%H%M")   # ex) 현재 시간이 14:15 인 경우, '1415' 출력
    tomorrow_date = today + datetime.timedelta(days=1)
    tomorrow_date = tomorrow_date.strftime("%Y%m%d") # 내일 날짜
    base_time = '0200'                    # 당일 02:00 발표시작으로 고정

    payload = "serviceKey=" + service_key + "&" +\
            "numOfRows=100" + "&" +\
            "pageNo=1" + "&" +\
            "dataType=json" + "&" +\
            "base_date=" + current_date + "&" +\
            "base_time=" + base_time + "&" +\
            "nx=" + nx + "&" +\
            "ny=" + ny
    # print(vilage_weather_url+payload)  # API url 확인

    # 값 요청
    res = requests.get(vilage_weather_url + payload)
    # res.status_code  # 200 정상 호출
    # res.json()
    items = res.json().get('response').get('body').get('items')

    for item in items['item']:
        if item['category'] == 'TMN':
            TMN = item['fcstValue']    
        elif item['category'] == 'TMN':
                    PTY = item['fcstValue']

    print(f"오늘 아침 최저기온은 {TMN}도 이고, 낮 최고기온은 {TMX}도 입니다.")

In [226]:
nx = '60'
ny = '128'
getTempMaxMinCurrent(nx, ny)

오늘 아침 최저기온은 15.0도 이고, 낮 최고기온은 26.0도 입니다.
